# Tranform the Github Data  

#### Read the data

In [1]:
# import the libraries
import pandas as pd
import numpy as np
import sys

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# read the data
df = pd.read_csv("../data/total_processed.csv", index_col=0)

FileNotFoundError: [Errno 2] File ../data/total_processed.csv does not exist: '../data/total_processed.csv'

In [4]:
df.shape

(601078, 63)

### Mean of the commit label

In [5]:
def mean(items):
    index = 0
    result = []
    value = 0

    for item in items:
        index = index + 1

        if item == 1:
            value = value + 1

        result.append((value/index)*100)

    return result

In [6]:
# create a dictionary project_name + mean_values for the commit classification
projects = df['name'].unique()
commit_mean_dict = {}

for project in projects:
    commits = df[df.name==project].results
    mean_commits = mean(commits)
    commit_mean_dict[project] = mean_commits

In [7]:
# replace all the data
df['mean_commits'] = 0

for project in projects:
    df.loc[df['name'] == project, 'mean_commits'] = commit_mean_dict[project]

### Variance of the column results

In [8]:
for project in projects:
    df.loc[df['name'] == project, 'variance_commits'] = df[df['name']==project][['results', 'mean_commits']].var(ddof=0, axis=1)

### Set the phases of the project

In [9]:
df = df.drop(df[df.sha == 'f5d7eb5b623b625062cf0d3d8d552ee0ea9000dd'].index)

In [10]:
def phases(entire_data):
    len_project = len(entire_data)
    phases = []
    
    end_phase_1 = len_project / 3
    end_phase_1 = int(end_phase_1)
    end_phase_2 = end_phase_1 + end_phase_1
    end_phase_2 = int(end_phase_2)

    i = 0

    for item in entire_data:
        if i < end_phase_1:
            phases.append(1)
            i = i + 1
        elif i >= end_phase_1 and i < end_phase_2:
            phases.append(2)
            i = i + 1
        else:
            phases.append(3)
            i = i + 1
        
    return phases

In [11]:
# create a dictionary project_name + phases for the commit classification
commit_phases_dict = {}

for project in projects:
    commits = df[df.name==project].results
    phases_commits = phases(commits)
    commit_phases_dict[project] = phases_commits

In [12]:
# replace all the data
df['phases_project'] = 0

for project in projects:
    df.loc[df['name'] == project, 'phases_project'] = commit_phases_dict[project]

In [13]:
df = pd.get_dummies(df, columns = ["phases_project"])

In [14]:
df[df['name']=='django']

,sha,message,comment_count,date,total_deletions,total_additions,total,comments,additions,deletions,changes,status,filenames,name,fullname,private,fork,size,watchers_count,has_issues,has_downloads,has_wiki,forks_count,open_issues_count,forks,open_issues,watchers,network_count,admin,push,pull,label,owner,language_C,language_C#,language_C++,language_CSS,language_CoffeeScript,language_Go,language_Java,language_JavaScript,language_PHP,language_Perl,language_Python,language_R,language_Ruby,language_Scala,language_Shell,language_TypeScript,type_Organization,type_User,total_files,results,date_only,is_weekend,is_night,outside_contribution,tests_included,most_changes_added,most_changes_modified,most_changes_removed,most_changes_renamed,most_changes_unknown,mean_commits,variance_commits,phases_project_1.0,phases_project_2.0,phases_project_3.0
9613,ed114e15106192b22ebb78ef5bf5bce72b419d13,Imported Django from private SVN repository (c...,0.0,2005-07-13T01:25:57Z,0.0,13851,13851,[],"[0, 0, 0, 15, 412, 0, 34, 22, 45, 36, 0, 0, 1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 15, 412, 0, 34, 22, 45, 36, 0, 0, 1,...","[u'added', u'added', u'added', u'added', u'add...",[u'backends/__init__.py b/django/core/db/backe...,django,django/django,0.0,0.0,125153.0,7404.0,0.0,1.0,0.0,2692.0,108.0,2692.0,108.0,7404.0,2692.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,114,0.0,Wednesday,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,0,0
9614,07ffc7d605cc96557db28a9e35da69bc0719611b,Imported fledgeling Django documentation from ...,0.0,2005-07-13T01:25:57Z,0.0,1885,1885,[],"[48, 258, 0, 0, 760, 804, 15]","[0, 0, 0, 0, 0, 0, 0]","[48, 258, 0, 0, 760, 804, 15]","[u'added', u'added', u'added', u'added', u'add...","[u'django-docs/build.py', u'django-docs/db-api...",django,django/django,0.0,0.0,125153.0,7404.0,0.0,1.0,0.0,2692.0,108.0,2692.0,108.0,7404.0,2692.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7,0.0,Wednesday,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,0,0
9623,d6ded0e91bcdd2a8f7a221f6a5552a33fe545359,Created basic repository structure\n\ngit-svn-...,0.0,2005-07-13T16:56:05Z,0.0,0,0,[],[],[],[],[],[],django,django/django,0.0,0.0,125153.0,7404.0,0.0,1.0,0.0,2692.0,108.0,2692.0,108.0,7404.0,2692.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,Wednesday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,1,0,0
9625,1cb1d9cd078adea3060742a768ef71f5bafb7f07,Moved django-docs to just plain docs\n\ngit-sv...,0.0,2005-07-13T18:54:07Z,0.0,0,0,[],"[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]","[u'renamed', u'renamed', u'renamed', u'renamed...","[u'docs/build.py', u'docs/db-api.txt', u'docs/...",django,django/django,0.0,0.0,125153.0,7404.0,0.0,1.0,0.0,2692.0,108.0,2692.0,108.0,7404.0,2692.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,7,0.0,Wednesday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000,1,0,0
9626,ce079538c66d826be5f0c4260ab4405b7abcbed7,Added basic structure for django repository\n\...,0.0,2005-07-13T19:08:56Z,0.0,78,78,[],"[1, 27, 1, 49]","[0, 0, 0, 0]","[1, 27, 1, 49]","[u'added', u'added', u'added', u'added']","[u'INSTALL', u'LICENSE', u'README', u'docs/faq...",django,django/django,0.0,0.0,125153.0,7404.0,0.0,1.0,0.0,2692.0,108.0,2692.0,108.0,7404.0,2692.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,0.0,Wednesday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,0,0
9630,3c0ee70e7c6b9e40cef611061f7756c80691ee76,Changed db/backends/postgresql.py to add the p...,0.0,2005-07-13T20:59:56Z,2.0,6,8,[],[6],[2],[8],[u'modified'],[u'django/core/db/backends/postgresql.py'],django,django/django,0.0,0.0,125153.0,7404.0,0.0,1.0,0.0,2692.0,108.0,2692.0,108.0,7404.0,2692.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.

#### Fill NAN values that can be filled with 0

In [15]:
df['comment_count'] = df['comment_count'].fillna(0)

#### Send the mean_commits to the end

In [16]:
# test the mean_commits as a regression problem
results = df.mean_commits.values
df = df.drop("mean_commits", axis=1)
df['mean_commits'] = results 

In [17]:
# save the csv
df.to_csv('../data/total_transformed.csv')